In [1]:
pip install names

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas==1.5.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import names
import random
import time
from datetime import datetime
import numpy as np
from random import randrange
from datetime import timedelta

In [4]:
#To generate initial timestamps

def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

Create product table

In [5]:
# initialize data of lists.
product_types = {'PID': ['P1', 'P2', 'P3'],
        'Value': [15,10,20],
         'Fragile': [0,0,1]}

# Create DataFrame
product_types_df = pd.DataFrame(product_types)

# Print the output.
product_types_df

,PID,Value,Fragile
0,P1,15,0
1,P2,10,0
2,P3,20,1


Create customer table

In [6]:
customers_df = pd.DataFrame(columns = ["CID", "Name","Bank Account"])
for i in range(50):
    customers_df = customers_df.append({'CID':"c"+str(i+1),
                                        'Name':names.get_full_name(),
                                        'Bank Account': 'BE'+ str(68-i)+" " + str(7689-i*3)+" "  + str(4567+i*13+67)+" "  + str(3467 - i*7 -789)},
                                       ignore_index=True)
customers_df

/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/3148138654.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  customers_df = customers_df.append({'CID':"c"+str(i+1),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/3148138654.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  customers_df = customers_df.append({'CID':"c"+str(i+1),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/3148138654.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  customers_df = customers_df.append({'CID':"c"+str(i+1),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/3148138654.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

,CID,Name,Bank Account
0,c1,Warren Pedraza,BE68 7689 4634 2678
1,c2,Robert Sleiman,BE67 7686 4647 2671
2,c3,Ryan Delangel,BE66 7683 4660 2664
3,c4,Patrica Wheeler,BE65 7680 4673 2657
4,c5,Eddie Perez,BE64 7677 4686 2650
5,c6,Jason Johnson,BE63 7674 4699 2643
6,c7,Paulette Valdez,BE62 7671 4712 2636
7,c8,Elizabeth Juarez,BE61 7668 4725 2629
8,c9,Annette Forshey,BE60 7665 4738 2622
9,c10,Robert Crockette,BE59 7662 4751 2615


Create Order table

In [7]:
orders_df = pd.DataFrame(columns = ["OID","Quantity"])

Create the events table and the dynamic attribute tables

In [8]:
events_df = pd.DataFrame(columns = ["EID", "Activity","Timestamp",'Customer','Order',"Product Type","Resource"])
quantity_df = pd.DataFrame(columns = ["QID", "P1","P2",'P3',"EID","OID"])
order_value_df = pd.DataFrame(columns = ["VID", "Value","EID","OID"])
refund_df = pd.DataFrame(columns = ["RID", "Refund Value","EID","OID"])
shipping_method_df = pd.DataFrame(columns = ["SID", "Method","EID","OID"])

Create list of potential resources for each activity

In [9]:
#Resources1 are responsible for the activities: Receive order, confirm purchase, determine shipping method,
#sent invoice, recollect letter, pay refund, file order

Resources1= ['Jan','Gabriel','Olivia','Sophia','Evelyn','Noah']

#Resources 2 are responsible for the activities: Collect product, put protection around product, add product to package

Resources2 = ['Johannes','Elizabeth','Jack','Chloe','Sebastian','Grace']

#Resources 3 are responsible for the activities: Ship by courrier, Ship by mail, ship by express courrier, deliver package

Resources3 = ['Alexandre','Violet','David','Zoe','Ryan','Claire']

In [10]:
def choice_excluding(lst, exception):
    possible_choices = [v for v in lst if v != exception]
    return random.choice(possible_choices)
print(choice_excluding(Resources1,random.choice(Resources1)))

Jan


Create first event

In [11]:
#Spare  with the idea of having the timestamps overlap
count = 1
Number_of_orders=100
min_value = 100
refund_prob = 0.5
time_between_events = 700
next_event_timerange = 10000

resource1 = random.choice(Resources1)
resource2 = random.choice(Resources2)
resource3 = random.choice(Resources3)

start = datetime.strptime('1/1/2022 08:30 AM', '%d/%m/%Y %I:%M %p')
end =  datetime.strptime('1/1/2022 6:00 PM', '%d/%m/%Y %I:%M %p')

for i in range(0, Number_of_orders):
    #Initialize customers
    customer= customers_df.sample()
    CID=customer['CID'].values
    #Initialize products
    product = product_types_df.sample(n=3,replace=True)
    product_list = set(product.PID.values.tolist())
    print('product_list', product_list)

    #Initialize resources
    resource1 = choice_excluding(Resources1,resource1)
    resource2 = choice_excluding(Resources2,resource2)
    resource3 = choice_excluding(Resources3,resource3)

    print(start,end)

    stamp = random_date(start, end)

# Create first event

    #Events Table
    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Place Order',
                                    'Timestamp': stamp,
                                 'Customer': CID[0],
                                 'Order': 'o'+str(i+1),
                                 'Product Type': product_list},
                                           ignore_index=True)
    # Order Table
    orders_df=orders_df.append({'OID':'o'+str(i+1)},ignore_index=True)
    # Quantity Table
    quantity_df = quantity_df.append({'QID': 'Q' + str(i+1),'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)
    total = 0
    Quantity = []
    #for q in range(0,3):
    for q in range(0, len(product_types_df.columns)):
            quantity = random.randint(1, 4)
            Quantity.append(quantity)
            value = product.iloc[q]['Value']*quantity
            total = total + value
    product['Quantity'] = Quantity
    q_sum = product.groupby('PID').sum()
    print(q_sum)


    for value in product_list:
        quantity_df.at[i,value] = q_sum.at[value,'Quantity']


    # Refund Table
    refund_df = refund_df.append({'RID':'R'+ str(refund_df.shape[0] + 1), 'Refund Value' : 0,'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)

    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)



 # Create event 2

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Receive Order',
                                    'Timestamp': stamp,
                                 'Customer': CID[0],
                                 'Order': 'o'+str(i+1),
                                 'Product Type': product_list,
                                 'Resource': resource1},
                                           ignore_index=True)
    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

 # Create event 3

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Confirm Purchase',
                                    'Timestamp': stamp,

                                 'Order': 'o'+str(i+1),
                                 'Product Type': product_list,
                                 'Resource': resource1},
                                           ignore_index=True)

    # Value Table (created by confirm purchase)

    order_value_df = order_value_df.append({'VID':'V'+str(i+1),'Value': total,'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)

    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

 # Create event 4
    for value in product_list:
        events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Collect product from warehouse',
                                    'Timestamp': stamp,

                                 'Order': 'o'+str(i+1),
                                 'Product Type': set([value]),
                                 'Resource': resource2},
                                           ignore_index=True)
        # Increase count
        count = count + 1

      #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)

 # Create event 5
    for value in product_list:
        fragile = product_types_df.loc[product_types_df['PID'] == value]['Fragile'].values
        if fragile[0] == 1:
            events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Put protection around the product',
                                    'Timestamp': stamp,

                                 'Order': 'o'+str(i+1),
                                 'Product Type': set([value]),
                                 'Resource': resource2},
                                           ignore_index=True)
            # Increase count
            count = count + 1

            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

 # Create event 6
    for value in product_list:
        events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Add product to package',
                                    'Timestamp':  stamp,

                                 'Order': 'o'+str(i+1),
                                 'Product Type': set([value]),
                                 'Resource': resource2},
                                           ignore_index=True)
        # Increase count
        count = count + 1

        #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)

# Create event 7
    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Get confirmation',
                                  'Timestamp': stamp,
                                 'Customer': CID[0],
                                 'Order': 'o'+str(i+1)},
                                           ignore_index=True)
    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

# Create event 8
    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Confirm shipping information',
                                  'Timestamp': stamp,
                                 'Customer': CID[0],
                                 'Order': 'o'+str(i+1)},
                                           ignore_index=True)
    # Increase count
    count = count + 1

   #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

# Create event 9
    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Determine shipping method',
                                  'Timestamp': stamp,
                                 'Product Type': product_list,
                                 'Order': 'o'+str(i+1),
                                 'Resource': resource1},
                                           ignore_index=True)
   # Increase count
    count = count + 1

  #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    #make the shipping method table

    #We need to parallelize therefore we make a split with random so either first the courrier is
    #send or first the invoice is send
    split = random.randint(0, 1)

    #Shipping method comes first

    if split==1:
        print('Shipping method comes first')
        #make event Determine shipping method and shipping method event itself
        control=0
        for value in product_list:
            fragile = product_types_df.loc[product_types_df['PID'] == value]['Fragile'].values
            if fragile[0] == 1:
                control = 1
                shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-1),
                                                        'SID': 'S'+str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Courrier',
                                                        'OID':'o' + str(i+1)},
                                               ignore_index=True)

                #Add shipping event courrier

                now=datetime.now()
                events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by courrier',
                                      'Timestamp': stamp,

                                     'Order': 'o'+str(i+1),
                                     'Resource': resource3},
                                               ignore_index=True)
                # Increase count
                count = count + 1

                #Increase timestamp at random between 1 and time_between_events seconds
                time_duration = random.uniform(1,time_between_events)

                stamp = stamp + timedelta(seconds=time_duration)

                break

        if order_value_df.loc[i]['Value'] >= min_value and control !=1 :
            control = 1
            shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-1),
                                                        'SID': 'S' + str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Courrier',
                                                        'OID':'o' + str(i+1)},
                                                   ignore_index=True)
            #Add shipping event courrier
            now=datetime.now()
            events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by courrier',
                                      'Timestamp': stamp,

                                     'Order': 'o'+str(i+1),
                                                         'Resource': resource3},
                                               ignore_index=True)
            # Increase count
            count = count + 1

            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

        if control !=1:
            shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-1),
                                                        'SID': 'S'+str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Mail',
                                                        'OID':'o' + str(i+1)},
                                                   ignore_index=True)
            #Add shipping event mail
            events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by mail',
                                      'Timestamp': stamp,
                                     'Order': 'o'+str(i+1),
                                                         'Resource': resource3},
                                               ignore_index=True)
            # Increase count
            count = count + 1

            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

        # Create event send invoice
        events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Send invoice',
                                      'Timestamp': stamp,
                                     'Order': 'o'+str(i+1),
                                     'Resource': resource1},
                                               ignore_index=True)
        # Increase count
        count = count + 1

        #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)

    #Sent invoice comes first

    if split==0:
        print('Send invoice comes first')
         # Create event send invoice
        events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Send invoice',
                                      'Timestamp': stamp,
                                     'Order': 'o'+str(i+1),
                                     'Resource': resource1},
                                               ignore_index=True)
        # Increase count
        count = count + 1

        #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)

        #make event Determine shipping method and shipping method event itself
        control=0
        for value in product_list:
            fragile = product_types_df.loc[product_types_df['PID'] == value]['Fragile'].values
            if fragile[0] == 1:
                control = 1
                shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-2),
                                                        'SID': 'S'+str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Courrier',
                                                        'OID':'o' + str(i+1)},
                                               ignore_index=True)

                #Add shipping event courrier

                now=datetime.now()
                events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by courrier',
                                      'Timestamp': stamp,

                                     'Order': 'o'+str(i+1),
                                     'Resource': resource3},
                                               ignore_index=True)
                # Increase count
                count = count + 1

               #Increase timestamp at random between 1 and time_between_events seconds
                time_duration = random.uniform(1,time_between_events)

                stamp = stamp + timedelta(seconds=time_duration)

                break

        if order_value_df.loc[i]['Value'] >= min_value and control !=1 :
            control = 1
            shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-2),
                                                        'SID': 'S' + str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Courrier',
                                                        'OID':'o' + str(i+1)},
                                                   ignore_index=True)
            #Add shipping event courrier
            now=datetime.now()
            events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by courrier',
                                      'Timestamp': stamp,

                                     'Order': 'o'+str(i+1),
                                                         'Resource': resource3},
                                               ignore_index=True)
            # Increase count
            count = count + 1

           #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

        if control !=1:
            shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-2),
                                                        'SID': 'S'+str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Mail',
                                                        'OID':'o' + str(i+1)},
                                                   ignore_index=True)
            #Add shipping event mail
            now=datetime.now()
            events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by mail',
                                      'Timestamp': stamp,
                                     'Order': 'o'+str(i+1),
                                                         'Resource': resource3},
                                               ignore_index=True)
            # Increase count
            count = count + 1


            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

    # Create event deliver package
    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Deliver package',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer':CID[0],
                                                     'Resource': resource3},
                                           ignore_index=True)

    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    # Create event Determine satisfaction
        #randomly decide on whether the customer wants a refund or not
    r = random.uniform(0, 1)
    #r = random.randint(0, 1)

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Determine satisfaction',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer':CID[0]},
                                           ignore_index=True)

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    if r<refund_prob:
        print('Customer does not want refund')
        count = count + 1
        events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'File Order',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Resource': resource1
                                 },
                                           ignore_index=True)


        #Initialize the next timerange
        start = stamp - timedelta(seconds= next_event_timerange)
        end = stamp

        continue

    #if no refund is asked the process ends with file order

    #create event File order
    else:
        refund_df = refund_df.append({'RID':'R' + str(refund_df.shape[0] + 1), 'Refund Value' : 1,'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)

        print('Since refund is asked, we take the other path')

     # Increase count
    count = count + 1

   #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    # Create event Request refund


    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Request refund',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer':CID[0]},
                                           ignore_index=True)


    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    # Create event Confirm shipping information

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Confirm shipping information',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer':CID[0]},
                                           ignore_index=True)

    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    # Create event Determine shipping method

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Determine shipping method',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Resource': resource1},
                                           ignore_index=True)


    shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count),
                                                    'SID': 'S'+ str(shipping_method_df.shape[0] + 1),
                                                    'Method': 'Express Courrier',
                                                    'OID':'o' + str(i+1)},
                                               ignore_index=True)

   # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)


    #We need to parallelize therefore we make a split with random so either first the express courrier is
    #send or first the recollect letter is send

    split = random.randint(0, 1)

    if split == 0:
            print('first express courrier then recollect letter')

            # Create event Ship by express courrier

            events_df = events_df.append({'EID':"e"+str(i+count),
                                          'Activity':'Ship by express courrier',
                                          'Timestamp': stamp ,
                                         'Order': 'o'+str(i+1),
                                                             'Resource': resource3},
                                                   ignore_index=True)




            # Increase count
            count = count + 1

            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

            # Create event Recollect Letter

            events_df = events_df.append({'EID':"e"+str(i+count),
                                          'Activity':'Recollect Letter',
                                          'Timestamp': stamp ,
                                         'Order': 'o'+str(i+1),
                                         'Resource': resource1},
                                                   ignore_index=True)


            # Increase count
            count = count + 1

            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

    if split == 1:
        print('first recollect letter then express courrier')

         # Create event Recollect Letter

        events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Recollect Letter',
                                      'Timestamp': stamp ,
                                     'Order': 'o'+str(i+1),
                                     'Resource': resource1},
                                               ignore_index=True)


        # Increase count
        count = count + 1

        #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)

        # Create event Ship by express courrier

        events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by express courrier',
                                      'Timestamp': stamp ,
                                     'Order': 'o'+str(i+1),
                                                         'Resource': resource3},
                                               ignore_index=True)

        # Increase count
        count = count + 1

        #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)



    # Create event Return Package

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Return Package',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer': CID[0]},
                                           ignore_index=True)


    # Increase count
    count = count + 1

    ##Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

     # Create event Pay refund

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Pay refund',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer': CID[0],
                                 'Resource': resource1},
                                           ignore_index=True)


    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    # Create event Deliver package

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Deliver package',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer': CID[0],
                                                     'Resource': resource3},
                                           ignore_index=True)


    # Increase count
    count = count + 1

   #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    #Create event Determine satisfaction

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Determine satisfaction',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer':CID[0]},
                                           ignore_index=True)

    #Change refund variable to 0 since the customer is happy

    refund_df = refund_df.append({'RID':'R' + str(refund_df.shape[0] + 1), 'Refund Value' : 0,'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)


   # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    #Create event File Order

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'File Order',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Resource': resource1},
                                           ignore_index=True)


    #Initialize the next timerange
    start = stamp- timedelta(seconds= next_event_timerange)
    end = stamp


product_list {'P2', 'P3'}
2022-01-01 08:30:00 2022-01-01 18:00:00
     Value  Fragile  Quantity
PID                          
P2      10        0         2
P3      40        2         7
Shipping method comes first
Since refund is asked, we take the other path
first recollect letter then express courrier
product_list {'P1', 'P2'}
2022-01-01 08:31:16.728503 2022-01-01 11:17:56.728503
     Value  Fragile  Quantity
PID                          
P1      15        0         4
P2      20        0         3
Send invoice comes first
Since refund is asked, we take the other path
first express courrier then recollect letter
product_list {'P1', 'P2'}
2022-01-01 10:02:38.509101 2022-01-01 12:49:18.509101
     Value  Fragile  Quantity
PID                          
P1      30        0         6
P2      10        0         4
Shipping method comes first
Customer does not want refund
product_list {'P2'}
2022-01-01 08:40:02.219563 2022-01-01 11:26:42.219563
     Value  Fragile  Quantity
PID              

/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  events_df = events_df.append({'EID':"e"+str(i+count),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  orders_df=orders_df.append({'OID':'o'+str(i+1)},ignore_index=True)
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quantity_df = quantity_df.append({'QID': 'Q' + str(i+1),'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:66: FutureWarning: The frame

     Value  Fragile  Quantity
PID                          
P1      45        0         7
Shipping method comes first
Since refund is asked, we take the other path
first express courrier then recollect letter
product_list {'P1', 'P3'}
2022-01-01 15:47:00.079222 2022-01-01 18:33:40.079222
     Value  Fragile  Quantity
PID                          
P1      15        0         4
P3      40        2         5
Send invoice comes first
Since refund is asked, we take the other path
first recollect letter then express courrier
product_list {'P1', 'P2', 'P3'}
2022-01-01 17:07:36.422926 2022-01-01 19:54:16.422926
     Value  Fragile  Quantity
PID                          
P1      15        0         3
P2      10        0         4
P3      20        1         2
Send invoice comes first
Customer does not want refund
product_list {'P1', 'P3'}
2022-01-01 18:17:31.327768 2022-01-01 21:04:11.327768
     Value  Fragile  Quantity
PID                          
P1      30        0         3
P3      20    

/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  refund_df = refund_df.append({'RID':'R'+ str(refund_df.shape[0] + 1), 'Refund Value' : 0,'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  events_df = events_df.append({'EID':"e"+str(i+count),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  events_df = events_df.append({'EID':"e"+str(i+count),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:109: Fut

Send invoice comes first
Customer does not want refund
product_list {'P2', 'P3'}
2022-01-02 06:13:57.693373 2022-01-02 09:00:37.693373
     Value  Fragile  Quantity
PID                          
P2      10        0         1
P3      40        2         6
Send invoice comes first
Customer does not want refund
product_list {'P1', 'P2', 'P3'}
2022-01-02 07:02:31.479232 2022-01-02 09:49:11.479232
     Value  Fragile  Quantity
PID                          
P1      15        0         4
P2      10        0         1
P3      20        1         1
Send invoice comes first
Customer does not want refund
product_list {'P1', 'P3'}
2022-01-02 07:49:59.240863 2022-01-02 10:36:39.240863
     Value  Fragile  Quantity
PID                          
P1      30        0         4
P3      20        1         3
Send invoice comes first
Customer does not want refund
product_list {'P3'}
2022-01-02 07:55:06.153400 2022-01-02 10:41:46.153400
     Value  Fragile  Quantity
PID                          
P3      60

...

/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:328: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  events_df = events_df.append({'EID':"e"+str(i+count),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:348: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-2),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:357: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  events_df = events_df.append({'EID':"e"+str(i+count),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:422: FutureWarning: The frame.append method is deprecated and will be removed from

Send invoice comes first
Customer does not want refund
product_list {'P1'}
2022-01-02 20:15:21.408510 2022-01-02 23:02:01.408510
     Value  Fragile  Quantity
PID                          
P1      45        0        10
Shipping method comes first
Since refund is asked, we take the other path
first express courrier then recollect letter
product_list {'P1', 'P2', 'P3'}
2022-01-02 20:08:39.729511 2022-01-02 22:55:19.729511
     Value  Fragile  Quantity
PID                          
P1      15        0         1
P2      10        0         1
P3      20        1         3
Send invoice comes first
Customer does not want refund
product_list {'P1', 'P2', 'P3'}
2022-01-02 19:44:46.000263 2022-01-02 22:31:26.000263
     Value  Fragile  Quantity
PID                          
P1      15        0         3
P2      10        0         2
P3      20        1         4
Shipping method comes first
Customer does not want refund
product_list {'P2', 'P3'}
2022-01-02 20:29:32.226368 2022-01-02 23:16:12.2263

/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:328: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  events_df = events_df.append({'EID':"e"+str(i+count),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:348: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-2),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:357: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  events_df = events_df.append({'EID':"e"+str(i+count),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:422: FutureWarning: The frame.append method is deprecated and will be removed from

product_list {'P1', 'P2'}
2022-01-02 19:24:08.840823 2022-01-02 22:10:48.840823
     Value  Fragile  Quantity
PID                          
P1      30        0         8
P2      10        0         4
Shipping method comes first
Customer does not want refund
product_list {'P2', 'P3'}
2022-01-02 18:50:30.101158 2022-01-02 21:37:10.101158
     Value  Fragile  Quantity
PID                          
P2      20        0         6
P3      20        1         1
Send invoice comes first
Customer does not want refund
product_list {'P1', 'P3'}
2022-01-02 18:51:36.755873 2022-01-02 21:38:16.755873
     Value  Fragile  Quantity
PID                          
P1      15        0         1
P3      40        2         6
Shipping method comes first
Since refund is asked, we take the other path
first express courrier then recollect letter
product_list {'P1', 'P2', 'P3'}
2022-01-02 19:27:54.086724 2022-01-02 22:14:34.086724
     Value  Fragile  Quantity
PID                          
P1      15        0   

/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  events_df = events_df.append({'EID':"e"+str(i+count),
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  orders_df=orders_df.append({'OID':'o'+str(i+1)},ignore_index=True)
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quantity_df = quantity_df.append({'QID': 'Q' + str(i+1),'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)
/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/2457804752.py:66: FutureWarning: The frame

Order the events by timestamps

In [12]:
#First order events by timestamps
events_df = pd.DataFrame(events_df)
events_df = events_df.sort_values(by=['Timestamp'])
#Reset the indexes
events_df= events_df.reset_index(drop=True)
events_df= events_df.reset_index()
#Rename column names
events_df=events_df.rename(columns = {'index':'EID_new', 'EID':'EID_old'})
#Rename the values of 'EID_new'
events_df['EID_new'] ='e' + events_df['EID_new'].astype(str)
events_df = events_df.fillna('/')
events_df.head(50)


,EID_new,EID_old,Activity,Timestamp,Customer,Order,Product Type,Resource
0,e0,e1,Place Order,2022-01-01 08:50:09.000000,c15,o1,"{P2, P3}",/
1,e1,e2,Receive Order,2022-01-01 08:53:24.674281,c15,o1,"{P2, P3}",Jan
2,e2,e3,Confirm Purchase,2022-01-01 09:00:47.833341,/,o1,"{P2, P3}",Jan
3,e3,e4,Collect product from warehouse,2022-01-01 09:07:09.417637,/,o1,{P2},Sebastian
4,e4,e5,Collect product from warehouse,2022-01-01 09:14:03.633403,/,o1,{P3},Sebastian
5,e5,e6,Put protection around the product,2022-01-01 09:21:59.768937,/,o1,{P3},Sebastian
6,e6,e7,Add product to package,2022-01-01 09:22:46.143840,/,o1,{P2},Sebastian
7,e7,e8,Add product to package,2022-01-01 09:32:59.535911,/,o1,{P3},Sebastian
8,e8,e9,Get confirmation,2022-01-01 09:39:10.662329,c15,o1,/,/
9,e9,e10,Confirm shipping information,2022-01-01 09:43:58.149743,c15,o1,/,/


In [13]:
#For every row in the dynamic attribute table
    #take the EID
    #look it up in the events_df table and retrieve the EID_new save it in a separate variable
    # put that value in the attributes table

for j in range(len(quantity_df)):
    ID = quantity_df.loc[j]['EID']
    quantity_df.loc[j]['EID'] = events_df['EID_new'].loc[events_df['EID_old'] == ID].values.tolist()[0]

for j in range(len(order_value_df)):
    ID = order_value_df.loc[j]['EID']
    order_value_df.loc[j]['EID'] = events_df['EID_new'].loc[events_df['EID_old'] == ID].values.tolist()[0]

for j in range(len(refund_df)):
    ID = refund_df.loc[j]['EID']
    refund_df.loc[j]['EID'] = events_df['EID_new'].loc[events_df['EID_old'] == ID].values.tolist()[0]

for j in range(len(shipping_method_df)):
    ID = shipping_method_df.loc[j]['EID']
    shipping_method_df.loc[j]['EID'] = events_df['EID_new'].loc[events_df['EID_old'] == ID].values.tolist()[0]

events_df= events_df.drop(['EID_old'], axis=1)
events_df.rename(columns = {'EID_new':'EID'}, inplace = True)



FIX QUANTITY TABLE


In [14]:
quantity_df[['P1','P2','P3']] =  quantity_df[['P1','P2','P3']].fillna(0)
quantity_df['Quantity']=quantity_df[['P1','P2','P3']].values.tolist()
quantity_df=quantity_df.drop(['P1','P2','P3'], axis=1)
columns_titles = ["QID","Quantity",'EID','OID']
quantity_df=quantity_df.reindex(columns=columns_titles)
quantity_df

,QID,Quantity,EID,OID
0,Q1,"[0, 2, 7]",e0,o1
1,Q2,"[4, 3, 0]",e21,o2
2,Q3,"[6, 4, 0]",e18,o3
3,Q4,"[0, 11, 0]",e28,o4
4,Q5,"[6, 0, 3]",e10,o5
...,...,...,...,...
95,Q96,"[0, 6, 1]",e1619,o96
96,Q97,"[1, 0, 6]",e1579,o97
97,Q98,"[3, 1, 4]",e1581,o98
98,Q99,"[0, 5, 3]",e1644,o99


In [15]:
orders_df["Quantity"]= quantity_df['Quantity']

In [16]:
orders_df

,OID,Quantity
0,o1,"[0, 2, 7]"
1,o2,"[4, 3, 0]"
2,o3,"[6, 4, 0]"
3,o4,"[0, 11, 0]"
4,o5,"[6, 0, 3]"
...,...,...
95,o96,"[0, 6, 1]"
96,o97,"[1, 0, 6]"
97,o98,"[3, 1, 4]"
98,o99,"[0, 5, 3]"


In [17]:
product_types_df

,PID,Value,Fragile
0,P1,15,0
1,P2,10,0
2,P3,20,1


In [18]:
customers_df

,CID,Name,Bank Account
0,c1,Warren Pedraza,BE68 7689 4634 2678
1,c2,Robert Sleiman,BE67 7686 4647 2671
2,c3,Ryan Delangel,BE66 7683 4660 2664
3,c4,Patrica Wheeler,BE65 7680 4673 2657
4,c5,Eddie Perez,BE64 7677 4686 2650
5,c6,Jason Johnson,BE63 7674 4699 2643
6,c7,Paulette Valdez,BE62 7671 4712 2636
7,c8,Elizabeth Juarez,BE61 7668 4725 2629
8,c9,Annette Forshey,BE60 7665 4738 2622
9,c10,Robert Crockette,BE59 7662 4751 2615


In [19]:
orders_df

,OID,Quantity
0,o1,"[0, 2, 7]"
1,o2,"[4, 3, 0]"
2,o3,"[6, 4, 0]"
3,o4,"[0, 11, 0]"
4,o5,"[6, 0, 3]"
...,...,...
95,o96,"[0, 6, 1]"
96,o97,"[1, 0, 6]"
97,o98,"[3, 1, 4]"
98,o99,"[0, 5, 3]"


In [20]:
quantity_df

,QID,Quantity,EID,OID
0,Q1,"[0, 2, 7]",e0,o1
1,Q2,"[4, 3, 0]",e21,o2
2,Q3,"[6, 4, 0]",e18,o3
3,Q4,"[0, 11, 0]",e28,o4
4,Q5,"[6, 0, 3]",e10,o5
...,...,...,...,...
95,Q96,"[0, 6, 1]",e1619,o96
96,Q97,"[1, 0, 6]",e1579,o97
97,Q98,"[3, 1, 4]",e1581,o98
98,Q99,"[0, 5, 3]",e1644,o99


In [21]:
order_value_df

,VID,Value,EID,OID
0,V1,160,e2,o1
1,V2,90,e30,o2
2,V3,130,e22,o3
3,V4,110,e38,o4
4,V5,150,e13,o5
...,...,...,...,...
95,V96,80,e1641,o96
96,V97,135,e1597,o97
97,V98,135,e1592,o98
98,V99,110,e1652,o99


In [22]:
refund_df

,RID,Refund Value,EID,OID
0,R1,0,e0,o1
1,R2,1,e26,o1
2,R3,0,e63,o1
3,R4,0,e21,o2
4,R5,1,e81,o2
...,...,...,...,...
191,R192,1,e1815,o99
192,R193,0,e1950,o99
193,R194,0,e1717,o100
194,R195,1,e1936,o100


In [23]:
shipping_method_df

,SID,Method,EID,OID
0,S1,Courrier,e11,o1
1,S2,Express Courrier,e42,o1
2,S3,Mail,e67,o2
3,S4,Express Courrier,e97,o2
4,S5,Courrier,e53,o3
...,...,...,...,...
143,S144,Express Courrier,e1864,o98
144,S145,Courrier,e1749,o99
145,S146,Express Courrier,e1892,o99
146,S147,Courrier,e1893,o100


In [24]:
events_df

,EID,Activity,Timestamp,Customer,Order,Product Type,Resource
0,e0,Place Order,2022-01-01 08:50:09.000000,c15,o1,"{P2, P3}",/
1,e1,Receive Order,2022-01-01 08:53:24.674281,c15,o1,"{P2, P3}",Jan
2,e2,Confirm Purchase,2022-01-01 09:00:47.833341,/,o1,"{P2, P3}",Jan
3,e3,Collect product from warehouse,2022-01-01 09:07:09.417637,/,o1,{P2},Sebastian
4,e4,Collect product from warehouse,2022-01-01 09:14:03.633403,/,o1,{P3},Sebastian
...,...,...,...,...,...,...,...
2032,e2032,Return Package,2022-01-03 01:06:10.601823,c23,o89,/,/
2033,e2033,Pay refund,2022-01-03 01:15:16.307116,c23,o89,/,Sophia
2034,e2034,Deliver package,2022-01-03 01:22:00.717702,c23,o89,/,Violet
2035,e2035,Determine satisfaction,2022-01-03 01:30:46.310957,c23,o89,/,/


In [25]:
writer = pd.ExcelWriter('DOCEL log.xlsx', engine='xlsxwriter')

In [26]:
# event table
events_df.to_excel(writer, sheet_name='Events')

#object tables
customers_df.to_excel(writer, sheet_name='Customer')
product_types_df.to_excel(writer, sheet_name='Product Type')
orders_df.to_excel(writer, sheet_name='Order')

#Attribute tables
#quantity_df.to_excel(writer, sheet_name='Quantity')
order_value_df.to_excel(writer, sheet_name='Value')
refund_df.to_excel(writer, sheet_name='Refund Value')
shipping_method_df.to_excel(writer, sheet_name='Method')

In [27]:
# Close the Pandas Excel writer and output the Excel file.
writer.save()

/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/1744656341.py:2: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [28]:
#OCEL LOG


In [32]:
#OCEL = pd.merge(, quantity_df, how="left",on='EID')
OCEL = pd.merge(events_df, order_value_df, how="left",on='EID')
OCEL = pd.merge(OCEL, refund_df, how="left",on='EID')
OCEL = pd.merge(OCEL, shipping_method_df, how="left",on='EID')
OCEL

,EID,Activity,Timestamp,Customer,Order,Product Type,Resource,VID,Value,OID_x,RID,Refund Value,OID_y,SID,Method,OID
0,e0,Place Order,2022-01-01 08:50:09.000000,c15,o1,"{P2, P3}",/,NaN,NaN,NaN,R1,0,o1,NaN,NaN,NaN
1,e1,Receive Order,2022-01-01 08:53:24.674281,c15,o1,"{P2, P3}",Jan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e2,Confirm Purchase,2022-01-01 09:00:47.833341,/,o1,"{P2, P3}",Jan,V1,160,o1,NaN,NaN,NaN,NaN,NaN,NaN
3,e3,Collect product from warehouse,2022-01-01 09:07:09.417637,/,o1,{P2},Sebastian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e4,Collect product from warehouse,2022-01-01 09:14:03.633403,/,o1,{P3},Sebastian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032,e2032,Return Package,2022-01-03 01:06:10.601823,c23,o89,/,/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2033,e2033,Pay refund,2022-01-03 01:15:16.307116,c23,o89,/,Sophia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2034,e2034,Deliver package,2022-01-03 01:22:00.717702,c23,o89,/,Violet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2035,e2035,Determine satisfaction,2022-01-03 01:30:46.310957,c23,o89,/,/,NaN,NaN,NaN,R175,0,o89,NaN,NaN,NaN


In [33]:

OCEL=OCEL.fillna('/')
OCEL=OCEL.drop(['OID_x','OID_y','VID','RID','SID'], axis=1)
OCEL

,EID,Activity,Timestamp,Customer,Order,Product Type,Resource,Value,Refund Value,Method,OID
0,e0,Place Order,2022-01-01 08:50:09.000000,c15,o1,"{P2, P3}",/,/,0,/,/
1,e1,Receive Order,2022-01-01 08:53:24.674281,c15,o1,"{P2, P3}",Jan,/,/,/,/
2,e2,Confirm Purchase,2022-01-01 09:00:47.833341,/,o1,"{P2, P3}",Jan,160,/,/,/
3,e3,Collect product from warehouse,2022-01-01 09:07:09.417637,/,o1,{P2},Sebastian,/,/,/,/
4,e4,Collect product from warehouse,2022-01-01 09:14:03.633403,/,o1,{P3},Sebastian,/,/,/,/
...,...,...,...,...,...,...,...,...,...,...,...
2032,e2032,Return Package,2022-01-03 01:06:10.601823,c23,o89,/,/,/,/,/,/
2033,e2033,Pay refund,2022-01-03 01:15:16.307116,c23,o89,/,Sophia,/,/,/,/
2034,e2034,Deliver package,2022-01-03 01:22:00.717702,c23,o89,/,Violet,/,/,/,/
2035,e2035,Determine satisfaction,2022-01-03 01:30:46.310957,c23,o89,/,/,/,0,/,/


In [34]:
writer = pd.ExcelWriter('OCEL log.xlsx', engine='xlsxwriter')

In [35]:
# event table
OCEL.to_excel(writer, sheet_name='Events')

#object tables
customers_df.to_excel(writer, sheet_name='Customer')
product_types_df.to_excel(writer, sheet_name='Product Type')
orders_df.to_excel(writer, sheet_name='Order')

In [36]:
# Close the Pandas Excel writer and output the Excel file.
writer.save()

/var/folders/3b/7cjmdk4d3ysg59q40b__n0mm0000gn/T/ipykernel_70534/1744656341.py:2: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
